In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error,explained_variance_score
import seaborn as sns
import kerastuner as kt

In [ ]:
call_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/Call_with_svol_vix.csv')
put_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/Put_with_svol_vix.csv')
call_df=call_df.set_index(['Date'])
put_df=put_df.set_index(['Date'])

In [ ]:
call_df.head()

In [ ]:
put_df.head()

In [ ]:
def regression(df):
  sns.scatterplot(x=df['Close'],y=df['bs_call'],hue=df['Expiration'])

In [ ]:
def split_data(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Adj Close','Expiration','Price','svol','Vix']],df['Close'],test_size=0.2,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [ ]:
def split_data_bs(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Adj Close','Expiration','Price','svol','Vix']],df['bs_call'],test_size=0.2,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [ ]:
n_units = 400
layers = 5
n_batch = 50
n_epochs = 20

In [ ]:
def make_model(hp):
    model = Sequential()
    model.add(Dense(n_units, input_dim=call_df.shape[1]-2))
    model.add(LeakyReLU())
    for i in range(hp.Int('num_layers',4,20)):
        model.add(Dense(hp.Int('units_'+str(i),min_value=200,max_value=800,step=50)))
        model.add(LeakyReLU())
        
    model.add(Dense(1, activation='relu'))
    opt = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-6, 1e-4,1e-5]))
    model.compile(loss='mae', optimizer=opt,metrics=['mse'])
    return model

In [ ]:
tuner = kt.Hyperband(
    make_model,
    objective='mse',
    directory='/content/drive/MyDrive/black_scholes_option_pricing',
    project_name='HyperParameters',
    max_epochs=20,seed=42,
    factor=3)

In [ ]:
tuner.search_space_summary()


In [ ]:
X_train, X_test, y_train, y_test=split_data(call_df)
tuner.search(X_train, y_train,epochs=20,validation_split=0.2)

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_model=tuner.hypermodel.build(best_hps)

In [ ]:
best_model.summary()

In [ ]:
def metrics(y_test,y_pred):
  x=np.array(y_test).reshape(-1,1)
  y=np.array(y_pred).reshape(-1,1)
  reg=LinearRegression().fit(x,y)
  mae= mean_absolute_error(x,y)
  mse=mean_squared_error(x,y)
  print("REG_SCORE - ",reg.score(x,y))
  print("Mean Absolute Error -",mae)
  print("Mean Squared Error -",mse)
  print("Explained_Variance_Score - ",explained_variance_score(x,y))
  print("")
  plt.scatter(x,y)
  print("")

In [ ]:
def prediction(df):
  X_train, X_test, y_train, y_test=split_data(df)
  X_train_bs, X_test_bs, y_train_bs, y_test_bs=split_data_bs(df)
  best_model.fit(X_train,y_train, epochs=20, validation_split=0.2)
  y_pred = best_model.predict(X_test)
  print("********Metrics of MLP MODEL********")
  metrics(y_test,y_pred)
  print("********Metrics of BlackScholes Model********")
  metrics(y_test,y_test_bs)

In [ ]:
prediction(call_df)

In [ ]:
prediction(put_df)